In [76]:
from GQ import TriGaussPoints3D

In [ ]:
import numpy as np
#import matplotlib.pyplot as plt
#from mpl_toolkits.mplot3d import Axes3D
import time
from scipy.sparse.linalg import cg
from scipy.sparse import coo_matrix

start_load = time.time()

step = 8

#Jacobian 행렬 구하기
ele = np.loadtxt(f'./step={step}/cube_Tri_h_{step}.txt', dtype = int)
ele = ele - 1
node = np.loadtxt(f'./step={step}/cube_Node_h_{step}.txt', dtype = np.float64)
w = TriGaussPoints3D(2)[:,-1]

end_load = time.time()
elapsed_time = end_load - start_load

print(f"LOAD 시간: {elapsed_time:.4f} 초")

start_total = time.time()

start_pre = time.time()

new = node[ele]
n = len(ele[0])
x1, y1, z1 = new[:,0,0], new[:,0,1], new[:,0,2]
x2, y2, z2 = new[:,1,0], new[:,1,1], new[:,1,2]
x3, y3, z3 = new[:,2,0], new[:,2,1], new[:,2,2]
x4, y4, z4 = new[:,3,0], new[:,3,1], new[:,3,2]

j11 = x2 - x1
j12 = x3 - x1
j13 = x4 - x1
j21 = y2 - y1
j22 = y3 - y1
j23 = y4 - y1
j31 = z2 - z1
j32 = z3 - z1
j33 = z4 - z1

#B를 (8,2,2) 3차원 벡터화 (순서대로 착착 잘 쌓아짐. 그대신 tran 안 하면 (2,2,8) 행렬이라 바꿔줌.)
B = np.array([[j11, j12, j13], [j21, j22, j23], [j31, j32, j33]]).transpose(2,0,1)

#det(B), |det(B)| 구하기
all_det = np.linalg.det(B)
det = abs(all_det)

#요소 강성행렬 구하기 위한
#B의 역행렬 구하기
B_inv = np.linalg.inv(B)
#그래디언트 파이 헷 펑션 저장하기
grad_phi = np.array([[-1,-1,-1], 
                    [1,0,0], 
                    [0,1,0],
                    [0,0,1]
])
#B_inv, 파이헷펑션의 행렬곱 미리 구해놓기
B_inv_x_phi =  grad_phi[np.newaxis,:,:] @ B_inv #grad_phi에 새로운축, 원래, 원래 3차로 바꿈

#요소 강성행렬의 요소 계산 중 내적값
A_ele = np.zeros((n,n,n))
A_ele = B_inv_x_phi @ B_inv_x_phi.transpose(0,2,1)
A_ele = A_ele * (det[:, np.newaxis, np.newaxis] / 6)

#---------------------------------

#*우변 f 구하기(b 구하기)

#f = 2 pi^2 * sin(pi x) sin(pi y) sin(pi * z) 정의하기
def f_func(x, y, z):
    return np.sin(np.pi * x) * np.sin(np.pi * y) * np.sin(np.pi * z) * 3 * (np.pi ** 2) 

#x, y값 모아두기
X = node[ele,0]
Y = node[ele,1]
Z = node[ele,2]

#자이, 에타, 크시 정의
xi = TriGaussPoints3D(2)[:,0]
yeta = TriGaussPoints3D(2)[:,1]
zeta = TriGaussPoints3D(2)[:,2]

#헷 파이 펑션 만들기
phi_func = np.array([1-xi-yeta-zeta, xi, yeta, zeta])

#x,y 치환
x_gp = X @ phi_func
y_gp = Y @ phi_func
z_gp = Z @ phi_func
        
        
#적분 전 안에 있는 f * 헷 파이 펑션 * |det| * w
f = f_func(x_gp, y_gp, z_gp) 
weights = det[:, np.newaxis] * w # |det| * w 계산 완료
val = np.zeros((len(ele), n))
val = (f * weights) @ phi_func.T   
#np.add.at(b, ind, out)이면 b의 ind번째 인덱스에 out이란 값 더함
b = np.zeros((len(node)))
np.add.at(b, ele.flatten(), val.flatten())
print(b)

#---------------------------------

end_pre = time.time()
elapsed_pre = end_pre - start_pre

print(f"Preprocessing 시간: {elapsed_pre:.4f} 초")

print(A_ele)

LOAD 시간: 0.0060 초
[6.01193283e-07 4.53465050e-05 1.32709093e-04 1.99867924e-04
 2.36598676e-04 2.37309424e-04 2.01891964e-04 1.35738282e-04
 4.95208705e-05 1.89079204e-05 3.64437065e-04 8.55514267e-04
 1.21634718e-03 1.39200226e-03 1.35573761e-03 1.11307420e-03
 7.00955337e-04 2.01030297e-04 4.44362098e-05 7.20576899e-04
 1.61905109e-03 2.27103943e-03 2.57728261e-03 2.49115787e-03
 2.02577693e-03 1.25198981e-03 3.32034805e-04 6.31994892e-05
 9.67015432e-04 2.13610207e-03 2.97998652e-03 3.37019505e-03
 3.24732192e-03 2.63007348e-03 1.61242018e-03 4.12490024e-04
 7.23412192e-05 1.06623463e-03 2.32795086e-03 3.23525768e-03
 3.65002584e-03 3.50911065e-03 2.83396518e-03 1.72737420e-03
 4.30147376e-04 7.04696544e-05 1.00312927e-03 2.16539024e-03
 2.99799018e-03 3.37417329e-03 3.23666910e-03 2.60641138e-03
 1.57935116e-03 3.82318689e-04 5.78697235e-05 7.87306578e-04
 1.67316859e-03 2.30430585e-03 2.58463344e-03 2.47147401e-03
 1.98205507e-03 1.19088621e-03 2.76285448e-04 3.64596518e-05
 4.516

In [78]:
start_assem = time.time()

#희소행렬 사용하여 A_matrix 조립
from scipy.sparse import coo_matrix
I = ele[:, :, np.newaxis].repeat(len(ele[0]), axis=2).flatten()
J = ele[:, np.newaxis, :].repeat(len(ele[0]), axis=1).flatten()
V = A_ele.flatten()
A = coo_matrix((V, (I, J)), shape=(len(node), len(node))).tocsr()

end_assem = time.time()
elapsed_assem = end_assem - start_assem

print(f"Assembly 시간: {elapsed_assem:.4f} 초")

print(A)


Assembly 시간: 0.0069 초
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9097 stored elements and shape (729, 729)>
  Coords	Values
  (0, 0)	0.06250000000000006
  (0, 1)	-0.020833333333333353
  (0, 9)	-0.020833333333333353
  (0, 81)	-0.020833333333333353
  (1, 0)	-0.020833333333333353
  (1, 1)	0.2083333333333335
  (1, 2)	-0.020833333333333353
  (1, 9)	-0.020833333333333353
  (1, 10)	-0.08333333333333341
  (1, 81)	-0.020833333333333353
  (1, 82)	-0.08333333333333341
  (1, 90)	0.041666666666666706
  (2, 1)	-0.020833333333333353
  (2, 2)	0.2083333333333335
  (2, 3)	-0.020833333333333353
  (2, 10)	-0.020833333333333353
  (2, 11)	-0.08333333333333341
  (2, 82)	-0.020833333333333353
  (2, 83)	-0.08333333333333341
  (2, 91)	0.041666666666666706
  (3, 2)	-0.020833333333333353
  (3, 3)	0.2083333333333335
  (3, 4)	-0.020833333333333353
  (3, 11)	-0.020833333333333353
  (3, 12)	-0.08333333333333341
  :	:
  (725, 716)	-0.08333333333333341
  (725, 717)	-0.020833333333333353
  (725, 724)	

In [79]:
import numpy as np

bd_idx = np.where(np.any(np.isin(node, [0,1]), axis=1))[0]
in_idx = np.where(~np.any(np.isin(node, [0,1]), axis=1))[0]

#A 행렬에 in index만 남도록 하기
A_reduced = A[in_idx, :][:, in_idx]
# 우변벡터(f)도 똑같이 in index만 남도록
f_reduced = b[in_idx]

from scipy.sparse.linalg import cg
# --------------------------------------------------------------------
# 1. 반복 횟수를 저장할 변수 초기화 및 콜백 함수 정의
it_count = 0

def callback_func(xk):
    global it_count
    it_count += 1

# 2. 시간 측정 시작 (CG 솔버 실행부터 결과 복원까지)
start_solving = time.time()

# 3. CG 솔버 실행 (spsolve 대신 오직 CG만 사용)
# tol=1e-6 인자를 추가하여 정밀도 설정
u_reduced, info = cg(A_reduced, f_reduced, rtol=1e-6, callback=callback_func)

# 4. 결과 확인 및 전체 노드로 복원
if info == 0:
    print(f"총 반복 횟수(Iteration): {it_count}")
    
    # 전체 노드 크기의 배열을 만들고 해 대입
    u_h = np.zeros(len(node))
    u_h[in_idx] = u_reduced
    
elif info > 0:
    print(f"수렴 실패. 현재까지 횟수: {it_count}")
else:
    print("에러 발생")
# --------------------------------------------------------------------
# 5. 시간 측정 종료 및 출력
end_solving = time.time()
end_total = time.time()

elapsed_solving = end_solving - start_solving
elapsed_total = end_total - start_total

print(f"CG Solve 실행 시간: {elapsed_solving :.4f} 초")
print(f"total: {elapsed_total}")

총 반복 횟수(Iteration): 15
CG Solve 실행 시간: 0.0010 초
total: 0.666395902633667


In [80]:
#L2 error 구하기
def u(x,y,z):
    return np.sin(np.pi * x) * np.sin(np.pi * y) * np.sin(np.pi * z)

phi_func = np.array([1-xi-yeta-zeta, xi, yeta, zeta])

#진짜 해 구하기
u_exact = u(node[:,0],node[:,1],node[:,2])
#에러 = 진짜 해 - 구한 해
error = u_exact - u_h

#M 구하기
phi_func_w = phi_func * w

#M 구하는 코드 벡터라이즈
i = np.repeat(ele,len(ele[0])).reshape(-1,len(ele[0]),len(ele[0])) #(n,4,4)
j = np.tile(ele,len(ele[0])).reshape(-1,len(ele[0]),len(ele[0])) #(n,4,4)
e = ((phi_func @ phi_func_w.T)[:,:,np.newaxis] @ det[np.newaxis,:]).transpose(2,0,1)
M_global = coo_matrix((e.flatten(), (i.flatten(), j.flatten())), shape=(len(node), len(node))).tocsr()

#L2 error 만들기
L2error = np.sqrt(error.T @ M_global @ error)
print(L2error)

0.014078567983908783


In [81]:
print('-'*17, '결과', '-'*17)
print(f"Preprocessing 시간 : {elapsed_pre:.4f} 초")
print(f"Assembly 시간      : {elapsed_assem:.4f} 초")
print(f"CG Solve 시간      : {elapsed_solving :.4f} 초")
print(f"Total 시간         : {elapsed_total}")
print(f"CG ITERATION       : {it_count}")
print(f"L2 error           : {L2error.item():.4e}")
print('-'*40)

----------------- 결과 -----------------
Preprocessing 시간 : 0.0272 초
Assembly 시간      : 0.0069 초
CG Solve 시간      : 0.0010 초
Total 시간         : 0.666395902633667
CG ITERATION       : 15
L2 error           : 1.4079e-02
----------------------------------------


In [82]:
#Convergence Rate
#계산할 때마다 바꾸기
def ConRate(E1, E2):
    return np.log2(E1 / E2)

print(ConRate(0.0009722875445736515,0.0002443835524709545
))

1.9922358479426086
